# The Travelling Salesman Problem
A classic optimization problem where the challenge is to find the shortest route connecting all nodes in a graph.  Brute force search methods require examining `(N-1)!` possible routes, which is infeasible for all but the smallest graphs.  The [A* algorithm](https://en.wikipedia.org/wiki/A*_search_algorithm) can be used to find the solve this in a more efficient manner; a simple, admissible heuristic for `A*` in this context is that the estimated distance to go for a partial route is equal to the shortest distance between any two nodes times the number of nodes left to visit.  See [wikipedia](https://simple.wikipedia.org/wiki/Travelling_salesman_problem) for more details on the history of the problem, and related graph search algorithms.

## Algorithm Implementation

In [132]:
import itertools
from Queue import PriorityQueue 

class Graph(object):
    def __init__(self, nodes=5):
        self.nodes = range(nodes)
        self.graph = np.random.random((nodes, 2))
        self.distances = {(i, j): np.linalg.norm(self.graph[i, :] - self.graph[j, :])
                          for i, j in itertools.combinations(self.nodes, 2)}

        for i, j in self.distances.keys():
            self.distances[j, i] = self.distances[i, j]
        
        for i in self.nodes:
            self.distances[i, i] = 0


class Path(object):
    def __init__(self, path, cost_so_far, heuristic):
        self.path = path
        self.cost_so_far = cost_so_far
        self.heuristic = heuristic
        self.total_cost = cost_so_far + heuristic
        
    def children(self, graph):
        heuristic = min(graph.distances.values()) * (len(graph.nodes) - len(self.path) - 1)
        return [Path(self.path + [step], 
                     self.cost_so_far + (graph.distances[self.path[-1], step] if self.path else 0), 
                     heuristic)
                for step in graph.nodes if step not in self.path]
    
    def __cmp__(self, other):
        return -1 if self.total_cost < other.total_cost else 1
        

class Astar(object):
    def __init__(self, graph):
        self.graph = graph
        self.best_path = None
        self.q = PriorityQueue()
        for seed in Path([], 0, 0).children(graph):
            self.q.put(seed)

    def next(self):
        if self.q.empty():
            return self.best_path
        
        p = self.q.get()
        if self.best_path and p.total_cost >= self.best_path.total_cost:
            self.q = PriorityQueue()
            return self.best_path
        
        if len(p.path) == len(self.graph.nodes):
            self.best_path = p
        else:
            for child in p.children(self.graph):
                self.q.put(child)
                
        return p

## Visualization

In [133]:
from matplotlib import animation
import animembed

graph = Graph(5)
astar = Astar(graph)
    
fig = plt.figure(figsize=(10, 6))
ax = plt.gca()
ax.set_axis_bgcolor('gainsboro')
ax.plot(graph.graph[:, 0], graph.graph[:, 1], 'r.')
plt.margins(.2, .2)
plt.xticks([])
plt.yticks([])
plt.axis('equal')
time_text = ax.text(0.01, 0.87, '', transform=ax.transAxes)

pair_to_line = {}
for c1, c2 in itertools.combinations(astar.graph.nodes, 2):
    line, = ax.plot(graph.graph[[c1, c2], 0], graph.graph[[c1, c2], 1], 'k')
    pair_to_line[tuple(sorted([c1, c2]))] = line


def animate(frame_no):
    # dim all the edges
    for line in pair_to_line.values():
        line.set_color('k')
        line.set_alpha(.1)
        
    # highlight all the edges in the current path
    p = astar.next()
    pairs = zip(p.path, p.path[1:])
    for pair in pairs:
        line = pair_to_line[tuple(sorted(pair))]
        line.set_color('b')
        line.set_alpha(1)
        
    time_text.set_text('\n'.join(['step: {}'.format(frame_no),
                                  'queue size: {}'.format(astar.q.qsize()),
                                  'path length: {:1.2f}'.format(p.total_cost)]))

def frames():
    frame = 0
    while not astar.q.empty():
        frame += 1
        yield frame
        
anim = animation.FuncAnimation(fig, animate, frames=frames, save_count=1<<32)
animembed.display_animation(anim, fps=10)

In [29]:
# save the gif if you like it!
anim.save('travelling_salesman.gif', writer='imagemagick', fps=15)